In [1]:
/******************************************************
 *
 * Name:         nb-make-refined-layer
 *     
 * Design Phase:
 *     Author:   John Miner
 *     Date:     03-10-2024
 *     Purpose:  Copy updated raw tables to stage [x].
 *               Update refined views to point to stage [x].
 *
 ******************************************************/

#### **Schema 1 - stage0**

In [2]:

--
-- Delete existing schema 
--

DROP SCHEMA IF EXISTS [stage0];
GO

--
-- Create new schema
--

CREATE SCHEMA [stage0];
GO


#### **Schema 2 - stage1**

In [3]:

--
-- Delete existing schema 
--

DROP SCHEMA IF EXISTS [stage1];
GO

--
-- Create new schema
--

CREATE SCHEMA [stage1];
GO


#### **Schema 3 - stage2**

In [4]:

--
-- Delete existing schema 
--

DROP SCHEMA IF EXISTS [stage2];
GO

--
-- Create new schema
--

CREATE SCHEMA [stage2];
GO


#### **Show Schemas + List DataSets**

In [7]:
select * from sys.schemas where name in ('raw', 'refined', 'stage0', 'stage1', 'stage2')

In [10]:
--
--  Show meta data for sets
--

select * from meta.DataSets

In [21]:
--
--  Show dynamic t-sql code
--

select * from meta.uvwCode4Staging

#### **Point in-time staging**

In [23]:
--
--  Del Procedure
--

DROP PROCEDURE IF EXISTS uspMakeStagingTables;
GO


--
--  Add Procedure
--

CREATE OR ALTER PROCEDURE uspMakeStagingTables 
    @VAR_STAGE INT = 1,
    @VAR_ALPHA INT = 1,
    @VAR_OMEGA INT = 12
AS
BEGIN

    -- While loop control
    DECLARE @VAR_CNT INT = @VAR_ALPHA;
    DECLARE @VAR_MAX INT = @VAR_OMEGA;

    -- Variables
    DECLARE @VAR_SCHEMA VARCHAR(32);    
    DECLARE @VAR_TABLE VARCHAR(128);
    DECLARE @VAR_FAIL VARCHAR(1024);
    DECLARE @VAR_MSG VARCHAR(128);

    -- Del + Add statements
    DECLARE @VAR_STMT1 NVARCHAR(4000);
    DECLARE @VAR_STMT2 NVARCHAR(4000);

    -- Dynamic schema name
    SELECT @VAR_SCHEMA = 'stage' + TRIM(STR(@VAR_STAGE));


    -- For each table
    WHILE (@VAR_CNT <= @VAR_MAX)
    BEGIN

        -- Grab code
        SELECT 
            @VAR_TABLE = [full_table_nm],
            @VAR_STMT1 = CAST(REPLACE([del_stmt], 'token', @VAR_SCHEMA) AS NVARCHAR(4000)),
            @VAR_STMT2 = CAST(REPLACE([add_stmt], 'token', @VAR_SCHEMA) AS NVARCHAR(4000))
        FROM 
            [meta].[uvwCode4Staging] 
        WHERE 
            [dataset_id] = @VAR_CNT;

        -- log start
        INSERT INTO [meta].[Logs]
        (
            job_id,
            step_id,
            program_name,
            step_description,
            action_type,
            additional_info,
            action_timestamp
        )
        VALUES
        (
            -1, 
            @VAR_CNT,
            'uspMakeStagingTables',
            're-create table - ' +  REPLACE(@VAR_TABLE, 'raw', @VAR_SCHEMA),
            'start tsql',
            'none',
            GetDate()
        );

        -- No failure yet
        SELECT @VAR_FAIL = 'none'

        -- Del + Add table
        BEGIN TRY
            
            -- Exec DEL code
            EXEC sp_executesql @VAR_STMT1;

            -- Exec ADD code
            EXEC sp_executesql @VAR_STMT2;

        END TRY

        BEGIN CATCH
            SELECT @VAR_FAIL = 'Error Num : ' + STR(ERROR_NUMBER()) + ', Error Msg: ' +  ERROR_MESSAGE();
        END CATCH


        -- log start
        INSERT INTO [meta].[Logs]
        (
            job_id,
            step_id,
            program_name,
            step_description,
            action_type,
            additional_info,
            action_timestamp
        )
        VALUES
        (
            -1, 
            @VAR_CNT,
            'uspMakeStagingTables',
            're-create table - ' +  REPLACE(@VAR_TABLE, 'raw', @VAR_SCHEMA),
            'end tsql',
            @VAR_FAIL,
            GetDate()
        );

        -- Increment counter
        SELECT @VAR_CNT = @VAR_CNT + 1;

    END;

END;
GO

In [24]:
--
--  Manual test of s.p.
--

uspMakeStagingTables 

In [25]:
select * from meta.Logs

In [27]:
select * from raw.Customer

#### **Refresh refined views**

In [28]:
--
--  Del Procedure
--

DROP PROCEDURE IF EXISTS uspRefreshRefinedViews;
GO


--
--  Add Procedure
--

CREATE OR ALTER PROCEDURE uspRefreshRefinedViews 
    @VAR_STAGE INT = 1,
    @VAR_ALPHA INT = 1,
    @VAR_OMEGA INT = 64    
AS
BEGIN

    -- While loop control
    DECLARE @VAR_CNT INT = @VAR_ALPHA;
    DECLARE @VAR_MAX INT = @VAR_OMEGA;

    -- Variables
    DECLARE @VAR_SCHEMA VARCHAR(32);
    DECLARE @VAR_TABLE VARCHAR(128);
    DECLARE @VAR_FAIL VARCHAR(1024);
    DECLARE @VAR_MSG VARCHAR(128);

    -- Del + Add statements
    DECLARE @VAR_STMT1 NVARCHAR(4000);
    
    -- Dynamic schema name
    SELECT @VAR_SCHEMA = 'stage' + TRIM(STR(@VAR_STAGE));

    -- For each table
    WHILE (@VAR_CNT <= @VAR_MAX)
    BEGIN

        -- Grab code
        SELECT 
            @VAR_TABLE = [full_table_nm],
            @VAR_STMT1 = CAST(REPLACE([upd_stmt], 'token', @VAR_SCHEMA) AS NVARCHAR(4000))
        FROM 
            [meta].[uvwCode4Staging] 
        WHERE 
            [dataset_id] = @VAR_CNT;

        -- log start
        INSERT INTO [meta].[Logs]
        (
            job_id,
            step_id,
            program_name,
            step_description,
            action_type,
            additional_info,
            action_timestamp
        )
        VALUES
        (
            -2, 
            @VAR_CNT,
            'uspRefreshRefinedViews',
            're-create view - ' + @VAR_TABLE,
            'start tsql',
            'none',
            GetDate()
        );

        -- No failure yet
        SELECT @VAR_FAIL = 'none'

        -- Upd Views
        BEGIN TRY
            
            -- Exec UPD code
            EXEC sp_executesql @VAR_STMT1;

        END TRY

        BEGIN CATCH
            SELECT @VAR_FAIL = 'Error Num : ' + STR(ERROR_NUMBER()) + ', Error Msg: ' +  ERROR_MESSAGE();
        END CATCH


        -- log start
        INSERT INTO [meta].[Logs]
        (
            job_id,
            step_id,
            program_name,
            step_description,
            action_type,
            additional_info,
            action_timestamp
        )
        VALUES
        (
            -2, 
            @VAR_CNT,
            'uspRefreshRefinedViews',
            're-create view - ' +  @VAR_TABLE,
            'end tsql',
            @VAR_FAIL,
            GetDate()
        );

        -- Increment counter
        SELECT @VAR_CNT = @VAR_CNT + 1;
    END;

END;
GO

In [ ]:
--
--  Manual test of s.p.
--

uspRefreshRefinedViews 


In [1]:
--
--  See Koen Verbeeck - https://tinyurl.com/y4x7u3wu
--

DECLARE @SqlStmt NVARCHAR(MAX);
SELECT @SqlStmt = COALESCE(@SqlStmt + ' UNION ALL ','')
    + 'SELECT '
    + '''' + QUOTENAME(SCHEMA_NAME(s.schema_id))
    + '.' + QUOTENAME(s.[name]) + '''' + ' AS [TableName], COUNT(*) AS [RowCount] FROM '
    + QUOTENAME(SCHEMA_NAME(s.schema_id))
    + '.' + QUOTENAME(s.[name])
FROM sys.objects AS s
WHERE s.[type] = 'U' AND s.is_ms_shipped = 0x0
ORDER BY SCHEMA_NAME(s.schema_id), s.[name];

EXEC sp_executesql @SqlStmt;

In [16]:
--
--  Verify incremental records have arrived
--

select ModifiedDate as ro, * from refined.SalesOrderHeader where ModifiedDate = '2025-06-08'
go
select ModifiedDate as ro, * from refined.SalesOrderDetail where ModifiedDate = '2025-06-01'
go


In [29]:
--
--  Inspect log entries
--

SELECT * FROM [meta].[Logs] order by action_timestamp desc

#### **SOD - Upsert incremental data into raw**

In [ ]:
--
--  SOD - Upsert data into table
--

-- drop old
DROP PROCEDURE IF EXISTS raw.uspMergeSalesOrderDetail;
GO

-- add new
CREATE PROCEDURE raw.uspMergeSalesOrderDetail
AS
BEGIN

    -- Ignore counts
    SET NOCOUNT ON;

    --
    -- Step 1: Update matching records in the TargetTable
	--
	
    UPDATE t
    SET 
		t.[SalesOrderID] = s.[SalesOrderID],
		t.[SalesOrderDetailID] = s.[SalesOrderDetailID],
		t.[OrderQty] = s.[OrderQty],
		t.[ProductID] = s.[ProductID],
		t.[UnitPrice] = s.[UnitPrice],
		t.[UnitPriceDiscount] = s.[UnitPriceDiscount],
		t.[LineTotal] = s.[LineTotal],
		t.[rowguid] = s.[rowguid],
		t.[ModifiedDate] = s.[ModifiedDate]	
    FROM 
	    raw.SalesOrderDetail as t
    INNER JOIN 
	    stage0.SalesOrderDetail as s
    ON 
	    t.SalesOrderDetailID = s.SalesOrderDetailID;

    --
    -- Step 2: Insert records into TargetTable that do not exist
	-- 
	
    INSERT INTO raw.SalesOrderDetail 
	(	
		[SalesOrderID],
		[SalesOrderDetailID],
		[OrderQty],
		[ProductID],
		[UnitPrice],
		[UnitPriceDiscount],
		[LineTotal],
		[rowguid],
		[ModifiedDate]
	)	
    SELECT
		s.[SalesOrderID],
		s.[SalesOrderDetailID],
		s.[OrderQty],
		s.[ProductID],
		s.[UnitPrice],
		s.[UnitPriceDiscount],
		s.[LineTotal],
		s.[rowguid],
		s.[ModifiedDate]
    FROM
        stage0.SalesOrderDetail as s
    LEFT JOIN 
	    raw.SalesOrderDetail as t
    ON 
	    t.SalesOrderDetailID = s.SalesOrderDetailID
    WHERE 
	    t.SalesOrderDetailID IS NULL;

END;
GO


#### **SOH - Upsert incremental data into raw**

In [ ]:
--
--  SOH - Upsert data into table
--

-- drop old
DROP PROCEDURE IF EXISTS raw.uspMergeSalesOrderHeader;
GO

-- add new
CREATE PROCEDURE raw.uspMergeSalesOrderHeader
AS
BEGIN

    -- Ignore counts
    SET NOCOUNT ON;

    --
    -- Step 1: Update matching records in the TargetTable
	--
	
    UPDATE t
    SET 
		t.[SalesOrderID] = s.[SalesOrderID],
		t.[RevisionNumber] = s.[RevisionNumber],
		t.[OrderDate] = s.[OrderDate],
		t.[DueDate] = s.[DueDate],
		t.[ShipDate] = s.[ShipDate],
		t.[Status] = s.[Status],
		t.[OnlineOrderFlag] = s.[OnlineOrderFlag],
		t.[SalesOrderNumber] = s.[SalesOrderNumber],
		t.[PurchaseOrderNumber] = s.[PurchaseOrderNumber],
		t.[AccountNumber] = s.[AccountNumber],
		t.[CustomerID] = s.[CustomerID],
		t.[ShipToAddressID] = s.[ShipToAddressID],
		t.[BillToAddressID] = s.[BillToAddressID],
		t.[ShipMethod] = s.[ShipMethod],
		t.[CreditCardApprovalCode] = s.[CreditCardApprovalCode],
		t.[SubTotal] = s.[SubTotal],
		t.[TaxAmt] = s.[TaxAmt],
		t.[Freight] = s.[Freight],
		t.[TotalDue] = s.[TotalDue],
		t.[Comment] = s.[Comment],
		t.[rowguid] = s.[rowguid],
		t.[ModifiedDate] = s.[ModifiedDate]
    FROM 
	    raw.SalesOrderHeader as t
    INNER JOIN 
	    stage0.SalesOrderHeader as s
    ON 
	    t.SalesOrderID = s.SalesOrderID;

    --
    -- Step 2: Insert records into TargetTable that do not exist
	-- 
	
    INSERT INTO raw.SalesOrderHeader 
	(	
		[SalesOrderID],
		[RevisionNumber],
		[OrderDate],
		[DueDate],
		[ShipDate],
		[Status],
		[OnlineOrderFlag],
		[SalesOrderNumber],
		[PurchaseOrderNumber],
		[AccountNumber],
		[CustomerID],
		[ShipToAddressID],
		[BillToAddressID],
		[ShipMethod],
		[CreditCardApprovalCode],
		[SubTotal],
		[TaxAmt],
		[Freight],
		[TotalDue],
		[Comment],
		[rowguid],
		[ModifiedDate]
	)
    SELECT
		s.[SalesOrderID],
		s.[RevisionNumber],
		s.[OrderDate],
		s.[DueDate],
		s.[ShipDate],
		s.[Status],
		s.[OnlineOrderFlag],
		s.[SalesOrderNumber],
		s.[PurchaseOrderNumber],
		s.[AccountNumber],
		s.[CustomerID],
		s.[ShipToAddressID],
		s.[BillToAddressID],
		s.[ShipMethod],
		s.[CreditCardApprovalCode],
		s.[SubTotal],
		s.[TaxAmt],
		s.[Freight],
		s.[TotalDue],
		s.[Comment],
		s.[rowguid],
		s.[ModifiedDate]
    FROM
        stage0.SalesOrderHeader as s
    LEFT JOIN 
	    raw.SalesOrderHeader as t
    ON 
	    t.SalesOrderID = s.SalesOrderID
    WHERE 
	    t.SalesOrderID IS NULL;

END;
GO

In [ ]:
--select * from stage0.SalesOrderDetail
-- exec raw.uspMergeSalesOrderDetail
exec raw.uspMergeSalesOrderHeader